In [ ]:
import queue
import time
import random
import heapq
import tracemalloc

def isValid (checkingState):
  for i in range(0,n-1):
    for j in range(i+1,n):
      if(checkingState[i] == checkingState[j]):
        return False
      if(abs(checkingState[i] - checkingState[j]) == j-i):
        return False
  return True

def calcHeuristic(checkingState):
  atkPairs = 0
  for i in range(0, n-1):
    for j in range(i+1, n):
      # 2 queens on the same diagonal or 2 queens on the same row
      if(checkingState[i] == checkingState[j]) or (abs(checkingState[i] - checkingState[j]) == j-i):
        atkPairs += 1
  return atkPairs

def presentResult(goalState):
  if(goalState != None):
    n = len(goalState)
    for i in range(0,n):
      for j in range(0,n):
        if(i == goalState[j]):
          print('Q', end = " ")
        else:
          print('*', end = " ")
      print()
  else:
    print("No solution")

# ----------------------------- UCS ----------------------------------------------

def UCS_Solution (n):
  cnt = 0
  frontier = queue.PriorityQueue()
  # create frontier with n element and cost = 0
  frontier.put((0,[0]*(n)))

  while(frontier.empty() == False or cnt<=n*10000):
    # get the state with minimal cost in frontier
    cost,curState = frontier.get()

    if(isValid(curState) == True):
      return curState

    for col in range (0,n):
        for row in range (0,n):
          nextState = curState.copy()
          # move a queen
          if(nextState[col] != row):
            nextState[col] = row
            # Each path has an unit cost
            nextCost = cost + 1
            frontier.put((nextCost, nextState))
    cnt +=1
  return None

# ------------------------- A* ----------------------------------------------

def AStar_Solution(n):
  frontier = queue.PriorityQueue()
  cnt = 0
  pathCost = 0
  cost = 0

  # this list will store states were checked
  checkedList = list()

  # create initial state: eight queen is on the same row
  frontier.put((calcHeuristic(([0]*n)), cost, [0]*n))
  while(frontier.empty() == False or cnt <= n*10000):
    h, c, curState = frontier.get()

    # wheather it is the goal state
    if(isValid(curState) == True):
      return curState

    # wheather it is in checked list
    if(curState in checkedList): continue

    # mark that this state is checked
    checkedList.append(curState)

    # find the successor
    successorList = list()
    for col in range(n):
      for row in range(n):
        if(row != curState[col]):
          nextState = curState.copy()
          nextState[col] = row
          successorList.append(nextState)

    # check and put successor into frontier
    for i in range(len(successorList)):
      if(successorList[i] not in checkedList):
        h = calcHeuristic(successorList[i])
        pathCost = h + c + 1
        frontier.put((pathCost, c+1, successorList[i]))
    cnt +=1
  return None

# -------------------------------- Genetic Algorithms ------------------
def calcFitness(checkingState):
  atkPairs = 0
  for i in range(0, n-1):
    for j in range(i+1, n):
      if(checkingState[i] == checkingState[j]) or (abs(checkingState[i] - checkingState[j]) == j-i):
        atkPairs += 1
  return n*(n-1)/2 - atkPairs

def createPopulation(size):
  population = list()
  for i in range (size):
    state = list()
    # random for the standing of queens
    for j in range(n):
      state.append(random.randint(0,n-1))
    population.append((calcFitness(state),state))
  return population

def chooseParents(population):
  # get the individual which has max/min fitness score
  par1 = max(population)
  par2 = min(population)
  population.remove(par2)
  return [par1, par2]

def crossOver(par1, par2):
  c = random.randint(0,n-1)
  if(par1.count(c)>1):
    k = par1.index(c) + 1
  elif(par2.count(c)>1):
    k = par2.index(c) + 1
  else:
    k = c
  child1 = par1[0:k] + par2[k:n]
  child2 = par2[0:k] + par1[k:n]
  return [child1, child2]

def mutation(child):
  # percentage for happening mutation
  if(random.random() < 0.8):
    curState = child.copy()
    number = random.randint(0,n-1)
    pos = random.randint(0,n-1)
    curState[pos] = number
    return curState
  return child

def GeneticAlgorithms_Solution(size, n):
  population = list()
  population = createPopulation(size)
  gens = 0
  while(gens <= n*10000):
    goalState = max(population)

    if(goalState[0] == n*(n-1)/2):
      return goalState[1]

    successorList = list()
    tmp = population.copy()
    count = 0

    while(len(tmp)>1):
      # return a list of parent
      parents = chooseParents(tmp)
      numChild = 0

      # return a list of child
      children = (list)(crossOver(parents[0][1],parents[1][1]))

      # each pair of parent must to cross-over successfully 2 child
      while(numChild < 2):
        for i in children:
          successor = mutation(i)
          if(successor not in [s[1] for s in population]):
            successorList.append((calcFitness(successor), successor))
            children.remove(i)
            numChild +=1

    population.clear()
    # choose successors that have high fitness score in successor list
    for i in range (size):
      maxNum = max(successorList)
      population.append(maxNum)
      successorList.remove(maxNum)
    gens+=1
  return None
# --------------------------------- main ---------------------
if __name__ == "__main__":
  choice = int(input("Number of algorithms: "))
  n = int(input("Number of queens: "))
  if(choice == 1):
    tracemalloc.start()
    startTime = time.time_ns()
    goalState = UCS_Solution(n)
    endTime = time.time_ns()
    p = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()
    print("--------------------------- UCS ---------------------------")
    print(goalState)
    presentResult(goalState)
    print("Total time :",(endTime-startTime)/10**6, "ms")
    print("Total memory: ", p / 1024**2, "MB")
  elif(choice == 2):
    tracemalloc.start()
    startTime = time.time_ns()
    goalState = AStar_Solution(n)
    endTime = time.time_ns()
    p = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()
    print("--------------------------- A* ---------------------------")
    print(goalState)
    presentResult(goalState)
    print("Total time :",(endTime-startTime)/10**6, "ms")
    print("Total memory: ", p / 1024**2, "MB")
  else:
    size = n
    tracemalloc.start()
    startTime = time.time_ns()
    goalState = GeneticAlgorithms_Solution(n,size)
    endTime = time.time_ns()
    p = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()
    print("--------------------------- GA ---------------------------")
    print(goalState)
    presentResult(goalState)
    print("Total time :",(endTime-startTime)/10**6, "ms")
    print("Total memory: ", p / 1024**2, "MB")



